# Практическая работа №2: Обработка выборочных данных. Нахождение точечных оценок параметров распределения
Выполнили студенты гр. 0381 Котов Дмитрий и Михайлов Виктор. Вариант №8

## Цель работы

Получение практических навыков нахождения точечных статистических оценок параметров распределения.

## Основные теоретические положения
Математическим ожиданием дискретной случайной величины называется сумма произведений ее возможных значений на соответствующие им вероятности:

$M(X) = \displaystyle \sum^n_{i=1} x_in_i$

Среднеквадратическим отклонением случайной величины $X$ (стандартом)
называется квадратный корень из ее дисперсии:

$\sigma = \sqrt{D(X)}$

Для упрощения вычислений эмперических моментов вводят в рассмотрение так называемые условные варианты:

$u_j = \frac{x_j - C}{h}$

где $C$ – условный ноль, значение которго выбирается равным значению интервального ряда, являющимся средним или близким к среднему по значению в этом ряду. В рузультате все условные варианты оказываются целыми числами.

Условные моменты $k$-го порядка:

$\displaystyle \bar{M}^*_k =\frac{1}{N} \sum n_j\left (\frac{x_j-C}{h}\right )^k = \frac{1}{N} \sum n_j u^k_j  $

Центральным моментом порядка $k$ случайной величины $X$ называется математическое ожидание величины:

$M(X-M(X))^k = m_k$

Исправленная выборочная дисперсия определяется по формуле:

$s^2 = \frac{N}{N-1}D_B$

где $\displaystyle D_B = \frac{1}{N}\sum^k_{i=1}(x_i-\bar{x})^2n_i$ – выборочная дисперсия

Асимметрией, или коэффициентом асимметрии, называется числовая характеристика, определяемая выражением:

$A = \frac{m_3}{s^3}$

где $m_3$ – центральный эмпирический момент третьего порядка, $s$ – стандартное отклонение.

Эксцессом называется численная характеристика случайной величины, которая определяется выражением:

$E = \frac{m_4}{s^4} - 3$

Для нормального закона $\frac{m_4}{s^4}$. Отсюда следует, что для нормального закона $E=0$. Эксцесс показывает, как быстро уменьшается плотность распределения вблизи её максимального значения.

Мода дискретной случайной величины – это наиболее вероятное значение
этой случайной величины. Модой непрерывной случайной величины называется
ее значение, при котором плотность вероятности максимальна.

$M_o(X) = x_{M_o} + h\frac{m_2 - m_1}{(m_2 - m_1)+(m_2-m_3)}$

Медиана случайной величины $X$ – это такое ее значение $M_e$, для которого выполнено равенство:

$P(X<M_e) = P(X>M_e)$

$M_e(X) = x_{M_e}+h\frac{0.5n-sM_{e^{-1}}}{n_{M_e}}$

## Постановка задачи

Для заданных выборочных данных вычислить с использованием метода моментов и условных вариант точечные статистические оценки математического ожидания, дисперсии, среднеквадратичного отклонения, асимметрии, эксцесса, моды, медианы и коэффициента вариации исследуемой случайной величины. Полученные результаты содержательно проинтерпретировать.

## Выполнение работы

### Импорт библиотек

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.stats import skew
from scipy.stats import kurtosis

### Восстановим результаты практической работы №1

In [11]:
data = pd.read_csv("/content/sample.csv")
data = data.sample(n=118, random_state=118)
data = data.sort_index()
data = data.reset_index(drop=True)
data.to_csv("/content/выборка.csv", index=False)

df = pd.read_csv("/content/выборка.csv")

df_nu = df.drop("E", axis=1)

df_nu = df_nu.sort_values(by=["nu"], ignore_index = True)

x = df_nu['nu']
var_series = pd.DataFrame({'nu': x.value_counts().sort_index().index})
var_series # Вариационный ряд

# Интервальный ряд
n=118
k = int(np.floor(1+3.31*np.log10(n))) # количество интервалов
h = int(np.ceil((max(x)-min(x))/k)) # ширина интервала
interval_bounds = [] # границы интервалов
x_min = min(var_series['nu'])
interval_bounds.append(x_min)
for i in range(1, k):
  interval_bounds.append(x_min+h*i)
interval_bounds.append(x_min+k*h)

counts = x.value_counts().sort_index().values # абсолютные частоты
rel_counts = x.value_counts(normalize=True).sort_index().values # относительные частоты

interval_series = pd.concat([var_series, pd.DataFrame({'n_i': counts, 'p_i': rel_counts})], axis=1)
interval_series["interval_i-i+1"] = pd.cut(interval_series["nu"], bins=interval_bounds, right=False)
interval_series = interval_series.groupby(["interval_i-i+1"])[["n_i", "p_i"]].apply(sum).reset_index()
interval_series["avg_i"] = np.array([np.mean([interval_bounds[i], interval_bounds[i+1]], axis=0) for i in range(k)])
interval_series = interval_series[["interval_i-i+1", "avg_i", "n_i", "p_i"]]

# Добавим накопленные частоты
n_cum = [] # накопленные абсолютные частоты
p_cum = [] # накопленные относительные частоты

n_abs, p_abs = interval_series['n_i'], interval_series['p_i']

for i in range(len(n_abs)):
  n_cum.append(sum(n_abs[:i+1]))
  p_cum.append(sum(p_abs[:i+1]))

interval_series['n_cum_i'] = n_cum
interval_series['p_cum_i'] = p_cum

### 1. Для середин интервального ряда, полученного в практической работе №1, вычислим условные варианты. Результаты представлены в табл.1.

In [25]:
# Вычислим условные варианты
C = interval_series["avg_i"][3]
u = [(x-C)/h for x in interval_series["avg_i"]]

# По методу произведений вычислим условные моменты
nu = [x*y for x,y in zip(u, interval_series['p_i'])]
nu2 = [(x**2)*y for x,y in zip(u, interval_series['p_i'])]
nu3 = [(x**3)*y for x,y in zip(u, interval_series['p_i'])]
nu4 = [(x**4)*y for x,y in zip(u, interval_series['p_i'])]
nuplus1_4 = [((x+1)**4)*y for x,y in zip(u, interval_series['p_i'])]

sum_nu = sum(nu)
sum_nu2 = sum(nu2)
sum_nu3 = sum(nu3)
sum_nu4 = sum(nu4)
sum_nuplus1_4 = sum(nuplus1_4)

Табл.1

| $i$      | $x_i$ | $n_i$    | $u_i$ | $n_iu_i$             | $n_iu_i^2$          | $n_iu_i^3$          | $n_iu_i^4$          | $n_i(u_i+1)^4$     |
|----------|-------|----------|-------|----------------------|---------------------|---------------------|---------------------|--------------------|
|     1    | 350.0 | 0.076271 |  -3.0 | -0.22881355932203387 |  0.6864406779661016 |  -2.059322033898305 | 6.177966101694914   | 1.2203389830508473 |
|     2    | 388.0 | 0.144068 |  -2.0 |   -0.288135593220339 |   0.576271186440678 |  -1.152542372881356 | 2.305084745762712   | 0.1440677966101695 |
|     3    | 426.0 | 0.245763 |  -1.0 |  -0.2457627118644068 |  0.2457627118644068 | -0.2457627118644068 | 0.2457627118644068  | 0.0                |
|     4    | 464.0 | 0.245763 |   0.0 |                  0.0 |                 0.0 |                 0.0 | 0.0                 | 0.2457627118644068 |
|     5    | 502.0 | 0.127119 |   1.0 |  0.12711864406779663 | 0.12711864406779663 | 0.12711864406779663 | 0.12711864406779663 | 2.033898305084746  |
|     6    | 540.0 | 0.118644 |   2.0 |  0.23728813559322035 |  0.4745762711864407 |  0.9491525423728814 | 1.8983050847457628  | 9.610169491525424  |
|     7    | 578.0 | 0.042373 |   3.0 |   0.1271186440677966 |  0.3813559322033898 |  1.1440677966101696 | 3.432203389830508   | 10.847457627118644 |
| $\Sigma$ |       |        1 |       | -0.27118644067796605 |  2.4915254237288136 | -1.2372881355932197 | 14.1864406779661    | 24.101694915254235 |

Произведём контроль вычислений по следующей формуле:
$$ \sum n_j (u_j + 1)^4 = \sum n_j u_j^4 + 4 \sum n_j u_j^3 + 6 \sum n_j u_j^2 + 4 \sum n_j u_j + \sum n_i $$

In [26]:
check_calc = sum_nu4 + 4*sum_nu3+6*sum_nu2+4*sum_nu+1
check_calc

24.10169491525424

Проверка пройдена

### 2. Вычислим условные эмпирические моменты через условные варианты и с помощью условных эмпирических моментов вычислим центральные эмпирические моменты. Полученные результаты представлены в табл. 2.

In [35]:
N = 7 # размер выборки

M1_star = sum_nu/N  # условный эмпирический момент 1-го порядка
M1 = M1_star*h + C  # эмпирический начальный момент

M2_star = sum_nu2/N  # условный эмпирический момент 2-го порядка
m2 = (M2_star - (M1_star**2))*(h**2)  # эмпирический центральный момент 2-го порядка

M3_star = sum_nu3/N  # условный эмпирический момент 3-го порядка
m3 = (M3_star - 3*M2_star*M1_star + 2*(M1_star**3))*(h**3)  # эмпирический центральный момент 3-го порядка

M4_star = sum_nu4/N  # условный эмпирический момент 4-го порядка
m4 = (M4_star - 4*M3_star*M1_star + 6*(M1_star**2)*M2_star - 3*(M1_star**4))*(h**4)  # эмпирический центральный момент 3-го порядка

M = [M1_star, M2_star, M3_star, M4_star]
m = [M1, m2, m3, m4]

for i in range(len(M)):
  print(f'Условный эмпирический момент {i+1}-го порядка: ', M[i])

for i in range(len(m)):
  print(f'Центральный эмпирический момент {i+1}-го порядка: ', m[i])

Условный эмпирический момент 1-го порядка:  -0.038740920096852295
Условный эмпирический момент 2-го порядка:  0.3559322033898305
Условный эмпирический момент 3-го порядка:  -0.17675544794188852
Условный эмпирический момент 4-го порядка:  2.0266343825665856
Центральный эмпирический момент 1-го порядка:  462.5278450363196
Центральный эмпирический момент 2-го порядка:  511.79886145782643
Центральный эмпирический момент 3-го порядка:  -7435.392723091459
Центральный эмпирический момент 4-го порядка:  4175364.2652651807


Табл.2

| $i$ | $v_i^*$               | $\mu_i^*$          |
|-----|-----------------------|--------------------|
| 1   | -0.038740920096852295 | 462.5278450363196  |
| 2   | 0.3559322033898305    | 511.79886145782643 |
| 3   | -0.17675544794188852  | -7435.392723091459 |
| 4   | 2.0266343825665856    | 4175364.2652651807 |

### 3. Вычислим выборочные среднее и дисперсию с помощью стандартной формулы и с помощью условных вариант. Убедимся, что результаты совпадают.

#### Выборочное среднее по стандартной формуле

In [29]:
x_mean1 = sum(interval_series["avg_i"]*interval_series["n_i"])/len(x)
x_mean1

453.6949152542373

С некоторой погрешностью совпадает с выборочным средним по условным вариантам

#### Дисперсия по стандартной формуле

In [39]:
xjv = [(x-x_mean1)**2 for x in interval_series["avg_i"]]
sum(interval_series["p_i"]*xjv)/N

498.79542003529366

С некоторой погрешностью совпадает с выборочной дисперсией по условным вариантам

### 4. Вычислим исправленную выборочную дисперсию и исправленное СКО. Сравнить данные оценки с смещёнными оценками дисперсии.

In [50]:
np.var(x, ddof = 1) # исправленная оценка дисперсии

3490.3825148486167

Выводы: поскольку оценка исправленная следовательно её мат. ожидание будет равно искомjму значению дисперсии

In [52]:
unbaised_msd = np.sqrt(np.var(x, ddof = 1)) # среднеквадратическое отклонение исправленной дисперсии
unbaised_msd

59.07945933104514

In [53]:
np.sqrt(np.var(x)) # среднеквадратическое отклонение смещённой дисперсии

58.828590005293485

### 5. Найдём статистическую оценку коэффициентов асимметрии и эксцесса.

In [54]:
asym = m[2]/pow(unbaised_msd,3) # асимметрия
asym

-0.036057397323424716

In [59]:
kurt = m[3]/pow(unbaised_msd,4) # Эксцес
kurt

0.34272700141313195

**Выводы:** асимметрия незначительна -> распределение близко к нормальному; эксцесс также незначителен, что свидетельсвтвует о хорошем соответствии нормальному распределению

### 6. Для интервального ряда вычислим моду, медиану и коэффициент вариации заданного распределения.

In [61]:
Moda = 407 + 39*((29-17)/(29-17+(29-29)))
Moda

446.0

In [62]:
Mediana = np.median(x)
Mediana

452.0

коэфф вариации = стандартное отклонение(СКО)/выборочное среднее

In [63]:
CV = unbaised_msd/x_mean1
CV

0.13021847357037

## Выводы

В ходе выполнения данной лабораторной работы были выполнены вычисления точечных оценок мат.ожидания, дисперсии, коэффициентов асимметрии и эксцесса методом моментов. Корректность используемого метода моментов была проверена с помощью сравнения значений, полученных с помощью условных вариант и стандартных формул. Были вычислены мода и медиана распределения. Оценены асимметрия и эксцесс.